In [ ]:
# Importar los Archivos Zip y descomprimir
from google.colab import files
uploaded = files.upload()

import zipfile
import os

# Cambia el nombre al de tu archivo
zip_filename = "archivo.zip"

# Crea una carpeta destino
output_dir = "Nombre de la Carpeta"
os.makedirs(output_dir, exist_ok=True)

# Extrae el contenido
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("¡Archivo descomprimido!")


In [ ]:
# Unificar datasets de anotaciones
import json
import os

#  Configurar
input_files = [
    "Jsons/Jsons/labels_orugas_coco.json",        # archivo con orugas
    "Jsons/Jsons/labels_grillos1_coco.json",     # archivo con grillos
    "Jsons/Jsons/labels_grillos2.json",
    "Jsons/Jsons/labels_grillos3.json"      # otro con grillos
]
output_file = "merged_dataset.json"

# Preparamos la estructura del archivo Coco.Json
merged = {
    "images": [],
    "annotations": [],
    "categories": []
}

# Se mapean las clases de los objetos
clase_a_id = {
    "grillos": 1,
    "oruga": 2
}

# Agregar categorías manualmente
merged["categories"] = [
    {"id": 1, "name": "grillo", "supercategory": "animal"},
    {"id": 2, "name": "oruga", "supercategory": "animal"}
]

# Contadores ID
image_id_offset = 0
annotation_id_offset = 0

for file_path in input_files:
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Mapear imágenes
    old_to_new_image_ids = {}
    for img in data["images"]:
        old_id = img["id"]
        new_id = image_id_offset
        img["id"] = new_id
        merged["images"].append(img)
        old_to_new_image_ids[old_id] = new_id
        image_id_offset += 1

    # Mapear anotaciones
    for ann in data["annotations"]:
        ann["id"] = annotation_id_offset
        ann["image_id"] = old_to_new_image_ids[ann["image_id"]]

        # Reasignar categoría si es necesario
        if isinstance(ann["category_id"], int):
            # Reasignar según nombre de clase si no confías en los ID originales
            class_name = next((c["name"] for c in data["categories"] if c["id"] == ann["category_id"]), None)
            ann["category_id"] = clase_a_id[class_name.lower()] if class_name else ann["category_id"]

        annotation_id_offset += 1
        merged["annotations"].append(ann)

# Guardamos
with open(output_file, "w") as f:
    json.dump(merged, f, indent=2)

print(f"Listo. Dataset unificado guardado como: {output_file}")

Listo. Dataset unificado guardado como: merged_dataset.json


In [ ]:
# Crear el dataset con todas las imágenes y anotaciones para entrenar
import json
import os
import shutil
import random

# Configuramos
json_path = "merged_dataset.json"
img_dir = "Imagenes"
output_dir = "dataset"
train_ratio = 0.8  # 80% entrenamiento

# Estructura de las carpeta
os.makedirs(f"{output_dir}/images/train", exist_ok=True)
os.makedirs(f"{output_dir}/images/val", exist_ok=True)
os.makedirs(f"{output_dir}/annotations", exist_ok=True)

# Cargamos Json
with open(json_path) as f:
    data = json.load(f)

images = data["images"]
annotations = data["annotations"]
categories = data["categories"]

# Hacemos un filtrado de las imágenes que no tienen etiquetas
ann_map = {}
for ann in annotations:
    ann_map.setdefault(ann["image_id"], []).append(ann)

images = [img for img in images if img["id"] in ann_map]

# Se divide para entrenamiento
random.shuffle(images)
train_count = int(len(images) * train_ratio)
train_images = images[:train_count]
val_images = images[train_count:]

train_image_ids = set(img["id"] for img in train_images)
val_image_ids = set(img["id"] for img in val_images)

# Filtrarmos anotaciones segun las imagenes
train_annotations = [ann for ann in annotations if ann["image_id"] in train_image_ids]
val_annotations = [ann for ann in annotations if ann["image_id"] in val_image_ids]

# Movemos las imagenes
for img in train_images:
    src = os.path.join(img_dir, img["file_name"])
    dst = os.path.join(output_dir, "images/train", img["file_name"])
    if os.path.exists(src):
        shutil.copyfile(src, dst)
    else:
        print(f"[ADVERTENCIA] Imagen no encontrada: {src}")

for img in val_images:
    src = os.path.join(img_dir, img["file_name"])
    dst = os.path.join(output_dir, "images/val", img["file_name"])
    if os.path.exists(src):
        shutil.copyfile(src, dst)
    else:
        print(f"[ADVERTENCIA] Imagen no encontrada: {src}")

# Guardamos json con las anotaciones
with open(f"{output_dir}/annotations/instances_train.json", "w") as f:
    json.dump({
        "images": train_images,
        "annotations": train_annotations,
        "categories": categories
    }, f, indent=2)

with open(f"{output_dir}/annotations/instances_val.json", "w") as f:
    json.dump({
        "images": val_images,
        "annotations": val_annotations,
        "categories": categories
    }, f, indent=2)

# Generamos el archivo data.yaml
class_names = {cat["id"]: cat["name"] for cat in categories}
sorted_classes = [class_names[i] for i in sorted(class_names)]

with open(f"{output_dir}/data.yaml", "w") as f:
    f.write(f"path: {output_dir}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n\n")
    f.write("names:\n")
    for i, name in enumerate(sorted_classes):
        f.write(f"  {i}: {name}\n")

print("\n Dataset preparado para entrenamiento con YOLOv8 segmentación.")
print(f"Dataset en: {output_dir}/")
print("Archivos importantes generados:")


[ADVERTENCIA] Imagen no encontrada: Imagenes/F9FNEM7YHUJC.jpg
[ADVERTENCIA] Imagen no encontrada: Imagenes/F17G43AKYEB0.jpg
[ADVERTENCIA] Imagen no encontrada: Imagenes/F98IS29OAN4S.jpg
[ADVERTENCIA] Imagen no encontrada: Imagenes/FBWAWC3WO0UG.jpg
[ADVERTENCIA] Imagen no encontrada: Imagenes/F9R56R1ZRF1R.jpg
[ADVERTENCIA] Imagen no encontrada: Imagenes/FBWYACTYV7J0.jpg
[ADVERTENCIA] Imagen no encontrada: Imagenes/FAITACR3OIFA.jpg
[ADVERTENCIA] Imagen no encontrada: Imagenes/F92ONF33IEXK.jpg

✅ ¡Listo! Dataset preparado para entrenamiento con YOLOv8 segmentación.
📁 Dataset en: dataset/
📄 Archivos importantes generados:
   - images/train/ y images/val/
   - annotations/instances_train.json y instances_val.json
   - data.yaml


In [ ]:
# Instalamos ultralytics
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import json
import os

def coco_segmentation_to_yolo(coco_json_path, images_dir, labels_dir):
    os.makedirs(labels_dir, exist_ok=True)

    with open(coco_json_path, 'r') as f:
        data = json.load(f)

    images = {img['id']: img for img in data['images']}
    categories = {cat['id']: idx for idx, cat in enumerate(data['categories'])}

    annotations_per_image = {}
    for ann in data['annotations']:
        img_id = ann['image_id']
        annotations_per_image.setdefault(img_id, []).append(ann)

    for img_id, img_info in images.items():
        img_width = img_info['width']
        img_height = img_info['height']
        filename = img_info['file_name']
        label_lines = []

        anns = annotations_per_image.get(img_id, [])
        for ann in anns:
            if 'segmentation' not in ann or not ann['segmentation']:
                continue
            category_id = ann['category_id']
            class_id = categories[category_id]

            # Algunas segmentaciones en COCO pueden tener múltiples polígonos (listas de listas)
            for segment in ann['segmentation']:
                # segment = [x1, y1, x2, y2, ..., xn, yn]
                if len(segment) < 6:
                    continue  # Polígonos inválidos (menos de 3 puntos)

                norm_coords = []
                for i in range(0, len(segment), 2):
                    x = segment[i] / img_width
                    y = segment[i + 1] / img_height
                    norm_coords.extend([f"{x:.6f}", f"{y:.6f}"])

                label_line = f"{class_id} " + " ".join(norm_coords)
                label_lines.append(label_line)

        label_filename = os.path.splitext(filename)[0] + ".txt"
        with open(os.path.join(labels_dir, label_filename), 'w') as f:
            f.write("\n".join(label_lines))

    print(f"Conversión de segmentación completada en {labels_dir}")

# USO:

coco_segmentation_to_yolo(
    coco_json_path='dataset/annotations/instances_train.json',
    images_dir='dataset/images/train',
    labels_dir='dataset/labels/train'
)

coco_segmentation_to_yolo(
    coco_json_path='dataset/annotations/instances_val.json',
    images_dir='dataset/images/val',
    labels_dir='dataset/labels/val'
)


Conversión de segmentación completada en dataset/labels/train
Conversión de segmentación completada en dataset/labels/val


In [ ]:
# Entrenamos el modelo (Usando CPU) más lejos
!yolo task=segment mode=train model=yolov8s-seg.pt data=dataset/data.yaml epochs=50 imgsz=640

Ultralytics 8.3.166 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pro

In [ ]:
# Entramos con GPU (Más rápido)
!yolo task=segment mode=train model=yolov8s-seg.pt data=dataset/data.yaml epochs=50 imgsz=416 save=true device=0

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 22.8M/22.8M [00:00<00:00, 99.3MB/s]
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=

In [ ]:
# Descargamos el mejor modelo
from google.colab import files
files.download('runs/segment/train/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Descargamos otros archivos útiles
files.download('runs/segment/train/weights/last.pt')
files.download('runs/segment/train/results.png')
files.download('runs/segment/train/labels.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>